**Exploratory Deep Learning Model – Medical Insurance Cost Prediction**


This notebook explores the use of a simple Deep Learning model (Feed-Forward Neural Network)
to predict medical insurance charges.

The goal is **not** to replace the production model, but to:
- Benchmark performance against classical ML approaches
- Evaluate trade-offs between accuracy, complexity, and interpretability
- Justify final model selection from an engineering and business perspective

In [1]:
#===========================================
#Importaciones
#===========================================

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


c:\Users\zukan\Desktop\metlife_ml\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
#=======================================
#Carga de datos
#=======================================
df = pd.read_csv("../data/dataset.csv")
df.head()



,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
#===============================================
#Definición de features y target
#===============================================

X = df.drop(columns=["charges"])
y = df["charges"]


In [5]:
#==================================
#Train / Test Split
#==================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [6]:
#=============================
#Preprocesamiento
#=============================  
numeric_features = ["age", "bmi", "children"]
categorical_features = ["sex", "smoker", "region"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),   # 👈 IMPORTANTE
        ("cat", OneHotEncoder(drop="first"), categorical_features)
    ]
)


In [7]:
#===============================
#Aplicar preprocesamiento
#===============================

X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.transform(X_test)

X_train_prep.shape, X_test_prep.shape


((1070, 8), (268, 8))

In [8]:
#======================================
#Definición del modelo de Deep learning
#======================================

model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train_prep.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss="mse"
)

model.summary()


c:\Users\zukan\Desktop\metlife_ml\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#================================
#Entrenamiento con Early Stopping
#==============================

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train_prep,
    y_train,
    validation_split=0.2,
    epochs=40,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 324757536.0000 - val_loss: 307710112.0000
Epoch 2/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 312884864.0000 - val_loss: 282916544.0000
Epoch 3/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 258777216.0000 - val_loss: 203492656.0000
Epoch 4/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 155730272.0000 - val_loss: 115476904.0000
Epoch 5/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 102293760.0000 - val_loss: 98765496.0000
Epoch 6/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 90265568.0000 - val_loss: 89946904.0000
Epoch 7/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 81106344.0000 - val_loss: 80210152.0000
Epoch 8/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 72568464.0000 - val_loss: 71420824.0000
Epoch 9/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 64431788.0000 - val_loss: 62776416.0000
Epoch 10/40
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 56841140.0000 - val_loss: 54645172.0000
Epoch 11/

In [10]:
#=======================================
#Evaluación del modelo
#========================================

y_pred_nn = model.predict(X_test_prep).ravel()

rmse_nn = mean_squared_error(y_test, y_pred_nn) ** 0.5
r2_nn = r2_score(y_test, y_pred_nn)

rmse_nn, r2_nn


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


(5235.1318336193335, 0.8234664782634519)

In [11]:
#======================================
#Comparación con Random Forest
#========================================

results = pd.DataFrame({
    "Modelo": ["Random Forest", "Red Neuronal"],
    "RMSE": [4576, rmse_nn],
    "R2": [0.865, r2_nn]
})

results


,Modelo,RMSE,R2
0,Random Forest,4576.000000,0.865000
1,Red Neuronal,5235.131834,0.823466


**Conclusión**

Se evaluó un modelo basado en redes neuronales como ejercicio exploratorio.
Sin embargo, los resultados obtenidos muestran que el modelo Random Forest
supera a la red neuronal tanto en RMSE como en R².

Dado el tamaño del dataset y la naturaleza tabular de los datos, el enfoque
basado en Random Forest ofrece un mejor balance entre desempeño predictivo,
robustez, interpretabilidad y simplicidad operativa.

Por este motivo, Random Forest fue seleccionado como modelo final para el
pipeline productivo.
